In [1]:
import selenium
print(selenium.__version__)

4.15.2


In [2]:
import csv
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains


DRIVER_PATH = "C:\\Users\\petr.musil\\Desktop\\python\\edgedriver\\msedgedriver.exe"
service = Service(executable_path = DRIVER_PATH)
driver = webdriver.Edge(service=service)

try:
    nakladatelstvi = "https://www.comicsdb.cz/prehled-nakladatelstvi/1/"
    list_of_links = []
    for num in list(range(1,8)):
        driver.get(f'{nakladatelstvi}{num}')    
        driver.implicitly_wait(1)
        element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".card.text-center.p-2")))
        #arrow_next_page = driver.find_element(By.CSS_SELECTOR, ".fa.fa-angle-right")
        #ActionChains(driver).scroll_to_element(arrow_next_page).perform()
        #arrow_next_page.click()
        links = driver.find_element(by = By.CSS_SELECTOR, value = ".table-border-dashed").find_elements(by = By.TAG_NAME, value = "a")
        for link in links:
            list_of_links.append([link.get_attribute("href")])
finally:
    driver.quit()

filename = "output.csv"
with open(filename, "w", newline="") as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(list_of_links)